## **Big data analysis project report and code: Bad/Good client detection from credit card informations** 

**1- Problem Introduction :**


---



Credit score cards are a common risk control method in the financial industry. It uses personal information and data submitted by credit card applicants to predict the probability of future defaults and credit card borrowings. The bank is able to decide whether to issue a credit card to the applicant. Credit scores can objectively quantify the magnitude of risk.
 
Generally speaking, credit score cards are based on historical data. Once encountering large economic fluctuations. Past models may lose their original predictive power. Logistic model is a common method for credit scoring. Because Logistic is suitable for binary classification tasks and can calculate the coefficients of each feature. In order to facilitate understanding and operation, the score card will multiply the logistic regression coefficient by a certain value (such as 100) and round it.
 
At present, with the development of machine learning algorithms. More predictive methods such as Boosting, Random Forest, and Support Vector Machines have been introduced into credit card scoring. However, these methods often do not have good transparency. It may be difficult to provide customers and regulators with a reason for rejection or acceptance.

**2- Dataset :**


---



We used two datasets :
* application_record.csv (438557 records)
* credit_record.csv (1048575 records)

>application_record.csv contains appliers personal information, which you could use as features for predicting.

>credit_record.csv records users' behaviors of credit card.

First we imported the libraries we need

In [1]:
import pandas as pd
import numpy as np
from imblearn.combine import SMOTETomek
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
import matplotlib.pyplot as plt

Then we uploaded our data

In [2]:
%%time
app = pd.read_csv('/content/application_record.csv')
credit = pd.read_csv('/content/credit_record.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/application_record.csv'

Then we did some breif data exploratory on our two datasets of credit card records and application records.

* App:

> Visualisation





In [3]:
app

NameError: name 'app' is not defined

> Columns

In [ ]:
app.columns

> Shape

In [ ]:
app.shape

> Some informations

In [ ]:
app.info()

In [ ]:
app.shape

* Credit:

> Visualisation

In [ ]:
credit

> Shape

In [ ]:
credit.shape

> Some informations

In [ ]:
credit.info()

**3- Preprocessing and feature engineering :**


---


After the data exploratory we remarqued that there is some 
duplicated columns on our application records dataset so we droped them.




In [ ]:
app.drop_duplicates(subset = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
       'AMT_INCOME_TOTAL', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE',
       'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS'], keep = 'first', inplace = True)

And we also noticed that there is some features that are not useful such as FLAG_OWN_REALTY.

In [ ]:
app.drop('FLAG_OWN_REALTY', axis = 1, inplace = True)
app.columns

Also we had to do some feature engineering to to analyze length of time since initial approval of credit card shows number of past dues, paid off and no loan status, and we counted the number of past dues, paif offs and no loans.Finally we merged the results inside the column ID inside our pivot table.

In [ ]:
grouped = credit.groupby('ID')

pivot_tb = credit.pivot(index = 'ID', columns = 'MONTHS_BALANCE', values = 'STATUS')
pivot_tb['open_month'] = grouped['MONTHS_BALANCE'].min()
pivot_tb['end_month'] = grouped['MONTHS_BALANCE'].max()
pivot_tb['window'] = pivot_tb['end_month'] - pivot_tb['open_month']
pivot_tb['window'] += 1 # Adding 1 since month starts at 0.

pivot_tb['paid_off'] = pivot_tb[pivot_tb.iloc[:,0:61] == 'C'].count(axis = 1)
pivot_tb['pastdue_1-29'] = pivot_tb[pivot_tb.iloc[:,0:61] == '0'].count(axis = 1)
pivot_tb['pastdue_30-59'] = pivot_tb[pivot_tb.iloc[:,0:61] == '1'].count(axis = 1)
pivot_tb['pastdue_60-89'] = pivot_tb[pivot_tb.iloc[:,0:61] == '2'].count(axis = 1)
pivot_tb['pastdue_90-119'] = pivot_tb[pivot_tb.iloc[:,0:61] == '3'].count(axis = 1)
pivot_tb['pastdue_120-149'] = pivot_tb[pivot_tb.iloc[:,0:61] == '4'].count(axis = 1)
pivot_tb['pastdue_over_150'] = pivot_tb[pivot_tb.iloc[:,0:61] == '5'].count(axis = 1)
pivot_tb['no_loan'] = pivot_tb[pivot_tb.iloc[:,0:61] == 'X'].count(axis = 1)
pivot_tb['ID'] = pivot_tb.index

In [ ]:
pivot_tb[:10].plot.bar(ylim=0)
plt.show()

In [ ]:
pivot_tb.head(10)

We defined a function to do some useful quick feature engineering and data cleaning.We counted the number of familly members and children to get a new feature of overall familly members, Then we droped some unuseful columns.
We converted the features DAYS_BIRTH and DAYS_EMPLOYED to years because this format is more important for our case of study.
Finally, We handled categorical variables by choosing most important categories after doing some researches.

In [ ]:
def data_cleansing(data):
    # Adding number of family members with number of children to get overall family members.
    data['CNT_FAM_MEMBERS'] = data['CNT_FAM_MEMBERS'] + data['CNT_CHILDREN']
    dropped_cols = ['FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE',
       'FLAG_EMAIL','OCCUPATION_TYPE','CNT_CHILDREN']
    data = data.drop(dropped_cols, axis = 1)
    
    #converting birth years and days employed to years.
    data['DAYS_BIRTH'] = np.abs(data['DAYS_BIRTH']/365)
    data['DAYS_EMPLOYED'] = data['DAYS_EMPLOYED']/365 
    
    #Cleaning up categorical values to lower the count of dummy variables.
    housing_type = {'House / apartment' : 'House / apartment',
                   'With parents': 'With parents',
                    'Municipal apartment' : 'House / apartment',
                    'Rented apartment': 'House / apartment',
                    'Office apartment': 'House / apartment',
                    'Co-op apartment': 'House / apartment'}
              
    income_type = {'Commercial associate':'Working',
                  'State servant':'Working',
                  'Working':'Working',
                  'Pensioner':'Pensioner',
                  'Student':'Student'}
    education_type = {'Secondary / secondary special':'secondary',
                     'Lower secondary':'secondary',
                     'Higher education':'Higher education',
                     'Incomplete higher':'Higher education',
                     'Academic degree':'Academic degree'}
    family_status = {'Single / not married':'Single',
                     'Separated':'Single',
                     'Widow':'Single',
                     'Civil marriage':'Married',
                    'Married':'Married'}
    data['NAME_HOUSING_TYPE'] = data['NAME_HOUSING_TYPE'].map(housing_type)
    data['NAME_INCOME_TYPE'] = data['NAME_INCOME_TYPE'].map(income_type)
    data['NAME_EDUCATION_TYPE']=data['NAME_EDUCATION_TYPE'].map(education_type)
    data['NAME_FAMILY_STATUS']=data['NAME_FAMILY_STATUS'].map(family_status)
    return data

In [ ]:
cleansed_app = data_cleansing(app)

A ratio based method was used to create the target variable. For example, given a client with a time period of 60 months, if the client had paid off loan 40 times and was late 20 times, this would be considered a fairly good client given that there were more loans that were paid off on time compared to late payments. If a client had no loans throughout the initial approval of the credit card account, by default, this would be considered a good client as well. To identify a bad client, the number of past dues would exceed the number of loans paid off  or if the client only has past dues. It may be better to incorporate a set difference between number of paid off loans and number of past dues. Meaning, there needs to be a significant gap between paid off loans and past dues. If a person has 50 past dues and 51 paid off loans, based on the ratio method, this would be considered good. However the difference is only 1 and this may not be a good sign of a good client. For simplicity sake, I will not adjust the algorithm further and keep it at ratio decisioning. Code is also not optimal, adjustment may be needed for the code to compute faster.

In [ ]:
def feature_engineering_target(data):
    good_or_bad = []
    for index, row in data.iterrows():
        paid_off = row['paid_off']
        over_1 = row['pastdue_1-29']
        over_30 = row['pastdue_30-59']
        over_60 = row['pastdue_60-89'] 
        over_90 = row['pastdue_90-119']
        over_120 = row['pastdue_120-149'] + row['pastdue_over_150']
        no_loan = row['no_loan']
            
        overall_pastdues = over_1+over_30+over_60+over_90+over_120    
            
        if overall_pastdues == 0:
            if paid_off >= no_loan or paid_off <= no_loan:
                good_or_bad.append(1)
            elif paid_off == 0 and no_loan == 1:
                good_or_bad.append(1)
        
        elif overall_pastdues != 0:
            if paid_off > overall_pastdues:
                good_or_bad.append(1)
            elif paid_off <= overall_pastdues:
                good_or_bad.append(0)
        
        elif paid_off == 0 and no_loan != 0:
            if overall_pastdues <= no_loan or overall_pastdues >= no_loan:
                good_or_bad.append(0)

        else:
            good_or_bad.append(1)
                
        
    return good_or_bad

There is some data of clients in the credit data that intersect with the application data. the following is a merge between the two data frames given on the data of clients that exist in both data sets. Featured engineered additional columns from the credit data.

In [ ]:
target = pd.DataFrame()
target['ID'] = pivot_tb.index
target['paid_off'] = pivot_tb['paid_off'].values
target['#_of_pastdues'] = pivot_tb['pastdue_1-29'].values+ pivot_tb['pastdue_30-59'].values + pivot_tb['pastdue_60-89'].values +pivot_tb['pastdue_90-119'].values+pivot_tb['pastdue_120-149'].values +pivot_tb['pastdue_over_150'].values
target['no_loan'] = pivot_tb['no_loan'].values
target['target'] = feature_engineering_target(pivot_tb)
credit_app = cleansed_app.merge(target, how = 'inner', on = 'ID')
credit_app.drop('ID', axis = 1, inplace = True)

**4- Model Selection and Postprocessing :**


---


We began our preprocessing with splitting our data to 80% of training set and 20% for testing.

In [ ]:
x = credit_app[credit_app.drop('target', axis = 1).columns]
y = credit_app['target']
xtrain, xtest, ytrain, ytest = train_test_split(x,y, train_size = 0.8, random_state = 0)

We transformed our data using One Hot encoding on categorical data because it's the best transformation we could use and it has a better reputation with resulting a good trained model.

In [ ]:
def transfromation(train, test):
    ohe = OneHotEncoder(drop = 'if_binary')
    xtrain_cat = train.select_dtypes(['object'])
    xtest_cat = test.select_dtypes(['object'])
    
    train_dummies = pd.DataFrame(ohe.fit_transform(xtrain_cat).todense(), 
                              columns = ohe.get_feature_names(xtrain_cat.columns))
    test_dummies = pd.DataFrame(ohe.transform(xtest_cat).todense(), 
                                columns = ohe.get_feature_names(xtest_cat.columns))
    train_dummies.index = train.index
    test_dummies.index = test.index
    
    train_with_dummies = pd.concat([train,train_dummies], axis = 1)
    train_with_dummies.drop(xtrain_cat.columns, axis = 1, inplace = True)
    
    test_with_dummies = pd.concat([test,test_dummies], axis = 1)
    test_with_dummies.drop(xtest_cat.columns, axis = 1, inplace = True)
    
    return train_with_dummies, test_with_dummies

In [ ]:
xtrain_with_dummies, xtest_with_dummies = transfromation(xtrain,xtest)

In [ ]:
print(ytrain.value_counts())
print(ytest.value_counts())

We noticed that our datasets are not highly imbalanced so we don't have to use a SMOTE oversampling, And for our model selection we choosed random forest wich adds additional randomness to the model, while growing the trees. Instead of searching for the most important feature while splitting a node, it searches for the best feature among a random subset of features. This results in a wide diversity that generally results in a better model.


We used a grid search with a selection of attributes to evaluate our model and get the best results, also avoid overfitting or underfitting.

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 2)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

grid_rf = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf = RandomForestClassifier()
grid = RandomizedSearchCV(rf, grid_rf, cv = 5,verbose = True, n_jobs = -1)
grid.fit(xtrain_with_dummies,ytrain)
params = grid.best_params_
print(params)

**5- Conclusion :**


---



After tunning our parameters and getting the best of this laters we started our training of the best model, Then we did some test to get our model accuracy,f1 score and efficiency(ROC AUC).

In [ ]:
rf_grid = RandomForestClassifier(n_estimators = 200, min_samples_split = 5, min_samples_leaf = 2, max_features = 'sqrt', max_depth = 80, bootstrap = True, random_state = 0) 
rf.fit(xtrain_with_dummies, ytrain) 
predictions_test = rf.predict(xtest_with_dummies)
roc_auc_test = roc_auc_score(ytest,predictions_test) 
accuracy_test = accuracy_score(ytest,predictions_test)
f1_test = f1_score(ytest, predictions_test)

print('roc_auc test: ', roc_auc_test) 
print('accuracy test: ', accuracy_test) 
print('f1 test: ', f1_test)

